In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [3]:
model = torch.hub.load('../yolov5', 'pose', source='local')
model.to(device)

YOLOv5 🚀 2022-11-2 Python-3.9.12 torch-1.13.0 CUDA:0 (NVIDIA Graphics Device, 24248MiB)

Fusing layers... 
Model summary: 157 layers, 1765930 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  

In [4]:
cap = cv2.VideoCapture('../videos/video_partB.avi')

In [5]:
success, img = cap.read()
fno = 0

frames = []
traces = {}

with open('./traces.csv', 'w') as f:
    f.write('fno,xmin,ymin,xmax,ymax,confidence,cls\n')
    while success:
        success, img = cap.read()
        if not success: 
            break

        bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        results = model(bgr)

        xyxy = results.xyxy[0]

        for t in xyxy:
            pos = t[:4]
            confidence = t[4]
            cls = t[5]

            f.write(f'{fno},')
            for t in pos:
                f.write(f'{t.item()},')
            f.write(f'{confidence.item()},')
            f.write(f'{cls.item()}\n')

        fno += 1

In [6]:
cap.release()

In [7]:
df = pd.read_csv('./traces.csv')
df = df[df['confidence'] > 0.8]
df.to_csv('./traces.csv', index=False)